In [1]:
import pandas as pd
from sklearn.metrics import auc
from os.path import join
from evaluator import Evaluator, CropOffsetFilter
from training_models import Autoencoder, DenoisingAutoencoder, AutoencoderEnsemble
from monitor_models import Autocovariance, AugmentationCrossCovariances, RepresentationCrossCovariances
from metrics import Identity, Variance, CovarianceWeightedAverage
from directory import get_history_csv_path, initiate_dirs

2024-06-14 13:52:50.179872: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-14 13:52:50.184780: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-14 13:52:50.242401: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 13:52:51.382477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
EXP_NAME = "exp_test"
TRAIN_DATASETS = ["stead"]
TEST_DATASETS = ["stead"]
SPLITS = range(0, 5)
NUM_EPOCHS = 20
FILTERS = [CropOffsetFilter(3.0, 3.0, 30.0, 100.0)]
OUTPUT_FILE_NAME = "summary.csv"

In [3]:
initiate_dirs()

In [4]:
def calculate_roc_aucs(training_model_class, monitoring_model_class, train_dataset, test_dataset, split, method_params, metric):
    evaluator = Evaluator(EXP_NAME, training_model_class, monitoring_model_class, 
                          train_dataset, test_dataset, FILTERS, range(NUM_EPOCHS), split, method_params, metric)
                    
    roc_vectors = evaluator.get_roc_vectors()
    roc_aucs = []

    for epoch in range(NUM_EPOCHS):
        roc_aucs.append(auc(roc_vectors[epoch]["tpr"], roc_vectors[epoch]["fpr"]))

    return roc_aucs

def get_validation_losses(training_model_class, train_dataset, split):
    model = training_model_class()
    df_history = pd.read_csv(get_history_csv_path(EXP_NAME, model.name, train_dataset, split))
    return df_history.loc[:, "val_loss"]

def model_summary(training_model_class, monitoring_model_class, method_params, metric):
    evaluations = []
    for train_dataset in TRAIN_DATASETS:
        for test_dataset in TEST_DATASETS:
            for split in SPLITS:
                roc_aucs = calculate_roc_aucs(training_model_class, monitoring_model_class, train_dataset, test_dataset, split, method_params, metric)
                val_loss = get_validation_losses(training_model_class, train_dataset, split)
                evaluations.append({"Training dataset": [train_dataset]*NUM_EPOCHS, 
                                    "Testing dataset": [test_dataset]*NUM_EPOCHS, 
                                    "Split": [split]*NUM_EPOCHS, 
                                    "Epoch": range(NUM_EPOCHS), 
                                    "Validation loss": val_loss, 
                                    "ROC-AUC":roc_aucs})

    df = pd.DataFrame(evaluations)

    df.loc[:, "Training model"] = training_model_class().name
    df.loc[:, "Monitoring model"] = monitoring_model_class().name

    return df

def grand_summary():
    df_autoencoder_autocovariance = model_summary(Autoencoder, Autocovariance, 
                                                  {}, 
                                                  CovarianceWeightedAverage(input_param="fcov"))
    
    df_denoisingae_autocovariance = model_summary(DenoisingAutoencoder, Autocovariance, 
                                                  {}, 
                                                  CovarianceWeightedAverage(input_param="fcov"))
    
    df_autoencoder_augmentation_crosscov = model_summary(Autoencoder, AugmentationCrossCovariances, 
                                                         {"augmentations": 5, "std": 0.15, "knots": 4}, 
                                                         CovarianceWeightedAverage(input_param="fcov"))
    
    df_denoisingae_augmentation_crosscov = model_summary(DenoisingAutoencoder, AugmentationCrossCovariances, 
                                                         {"augmentations": 5, "std": 0.15, "knots": 4}, 
                                                         CovarianceWeightedAverage(input_param="fcov"))
    
    df_autoencoder_ensemble_representation_crosscov = model_summary(AutoencoderEnsemble, RepresentationCrossCovariances, 
                                                                    {}, 
                                                                    CovarianceWeightedAverage(input_param="fcov"))

    df_grand_summary = pd.concat(
        [
            df_autoencoder_autocovariance,
            df_denoisingae_autocovariance,
            df_autoencoder_augmentation_crosscov,
            df_denoisingae_augmentation_crosscov,
            df_autoencoder_ensemble_representation_crosscov,
        ],
        axis=1,
    )

    df_grand_summary.sort_values(
        by=[
            "Monitoring model",
            "Training model",
            "Training dataset",
            "Testing dataset",
            "Epoch",
            "Split",
        ],
        inplace=True,
    )

    return df_grand_summary

In [5]:
df = grand_summary()
df.to_csv(join(EXP_NAME, OUTPUT_FILE_NAME))

TypeError: Exception encountered when calling Autoencoder.call().

[1mtoo many positional arguments[0m

Arguments received by Autoencoder.call():
  • inputs=tf.Tensor(shape=(256, 3000, 3), dtype=float32)
  • training=False